In [1]:
#!/usr/bin/env python3

import os
import warnings

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances

#import tensorflow as tf
#import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
#import seaborn as sns

import torch
from transformers import BertModel
from transformers import BertTokenizer

def load_gold(filepath_or_buffer, sep='\t'):
    df = pd.read_csv(filepath_or_buffer, sep=sep, dtype=str)

    gold = OrderedDict()

    for _, row in df[['questionID', 'explanation']].dropna().iterrows():
        explanations = OrderedDict((uid.lower(), Explanation(uid.lower(), role))
                                   for e in row['explanation'].split()
                                   for uid, role in (e.split('|', 1),))

        question = Question(row['questionID'].lower(), explanations)

        gold[question.id] = question

    return gold

def read_explanations(path):
    header = []
    uid = None

    df = pd.read_csv(path, sep='\t', dtype=str)

    for name in df.columns:
        if name.startswith('[SKIP]'):
            if 'UID' in name and not uid:
                uid = name
        else:
            header.append(name)

    if not uid or len(df) == 0:
        warnings.warn('Possibly misformatted file: ' + path)
        return []

    return df.apply(lambda r: (r[uid], ' '.join(str(s) for s in list(r[header]) if not pd.isna(s))), 1).tolist()

In [2]:
import argparse

tables = 'data/annotation/expl-tablestore-export-2017-08-25-230344/tables'
questions = 'data/questions/ARC-Elementary+EXPL-Dev.tsv'
quesitons = open(questions, 'r')

explanations = []

for path, _, files in os.walk(tables):
    for file in files:
        explanations += read_explanations(os.path.join(path, file))

if not explanations:
    warnings.warn('Empty explanations')

# prepare data
df_q = pd.read_csv(questions, sep='\t', dtype=str)
text_q = [q for q in df_q['Question']]
df_e = pd.DataFrame(explanations, columns=('uid', 'text'))
text_e = [e for e in df_e['text']]

/home/haoyu/.local/lib/python3.5/site-packages/ipykernel_launcher.py:54: UserWarning: Possibly misformatted file: data/annotation/expl-tablestore-export-2017-08-25-230344/tables/LemmatizerAdditions.tsv
/home/haoyu/.local/lib/python3.5/site-packages/ipykernel_launcher.py:54: UserWarning: Possibly misformatted file: data/annotation/expl-tablestore-export-2017-08-25-230344/tables/Decompositions.tsv
/home/haoyu/.local/lib/python3.5/site-packages/ipykernel_launcher.py:54: UserWarning: Possibly misformatted file: data/annotation/expl-tablestore-export-2017-08-25-230344/tables/PROCESSSTAGES.tsv


In [3]:
uid_list = []
for i in df_e['uid']:
    uid_list.append(i)

In [4]:
'ac40-d9c4-86a2-f4bb' in uid_list

True

In [18]:
explanations = df_q['explanation'][1].strip().split('|')[:-1]

In [19]:
clean_exp = [explanations[0]] + [x.split()[1] for x in explanations[1:]]

In [35]:
set_e = {}
for i in range(len(df_e)):
    set_e[df_e['uid'][i]] = df_e['text'][i]

In [56]:
all_uids = list(set_e.keys())

In [57]:
all_uids

['36c1-973c-ac47-fe55',
 '5ede-59cb-d266-b4b0',
 '93fd-6a6f-7782-77d1',
 '89d4-504e-26ea-25da',
 'fd2d-7763-c40b-59a9',
 'd042-a628-d170-532f',
 '627d-87b0-4a4a-7cad',
 '4a5d-ae1d-5ba6-8f8d',
 '33c7-1992-4659-d284',
 'ed80-15ae-c493-8ca6',
 'a50c-ba38-bbe4-12c1',
 'd6fc-5f39-b200-7600',
 '26d4-7eb1-9874-3e50',
 '56bd-8c40-3dae-d3e9',
 '58fb-4d30-982b-787b',
 '52a3-125f-af5d-1f1c',
 'db13-7d7e-3c0b-55f1',
 'aa9b-a7b0-3501-e36c',
 '8eef-9fa6-ffe3-c079',
 'e2a9-870f-4d89-e4b8',
 '7a74-426d-ec77-f296',
 'f8b7-590f-a416-818a',
 'b5e7-965c-c67f-f9d9',
 '123d-8a65-3831-3fa1',
 '1d10-2a0a-bf2e-7622',
 'e58d-03a1-b94d-942d',
 'fba1-b7e2-ae8b-6182',
 'fd56-73df-1f6f-e1bc',
 '000b-8380-fb97-aee5',
 'efca-82f8-2845-8de5',
 '489a-467c-0138-cd62',
 '72a7-76e4-7f5d-03ab',
 '1039-d8f3-b11c-3d03',
 'a6f0-93ef-5850-b78f',
 'badc-0c72-9633-5872',
 'bbe2-8087-7d0d-2480',
 'b4d1-4fcf-d14e-cb3b',
 'c546-7b47-1430-cf87',
 '8fd1-cc47-8c1f-bbdb',
 '24c0-2337-1514-44da',
 'eb54-6da1-4c1e-fb6a',
 '7cc2-cadd-f58b

In [61]:
import random

In [65]:
random_uid = all_uids[random.randint(0, len(text_e))]
print(random_uid in clean_exp)

False


In [53]:
q_e_pair = []
for i in range(len(text_q)):
    question_text = text_q[i]
    explanations = df_q['explanation'][i].strip().split('|')[:-1]
    clean_exp = [explanations[0]] + [x.split()[1] for x in explanations[1:]]
    explanation_texts = [set_e[id] for id in clean_exp]
    
    random_explanation = 
    for explanation in explanation_texts:
        q_e = question_text + ' [SEP] ' + explanation
        
        token_q_e = tokenizer.encode(q_e)
        print(token_q_e)
    break

Which of these is MOST flexible? (A) Broom handle (B) Wooden ruler (C) Drinking straw (D) Sewing needle [SEP] flexible means high flexibility
[101, 2029, 1997, 2122, 2003, 2087, 12379, 1029, 1006, 1037, 1007, 23528, 5047, 1006, 1038, 1007, 4799, 7786, 1006, 1039, 1007, 5948, 13137, 1006, 1040, 1007, 22746, 12201, 102, 12379, 2965, 2152, 16991, 102]
Which of these is MOST flexible? (A) Broom handle (B) Wooden ruler (C) Drinking straw (D) Sewing needle [SEP] flexibility means the shape of a material; of an object can be easily changed
[101, 2029, 1997, 2122, 2003, 2087, 12379, 1029, 1006, 1037, 1007, 23528, 5047, 1006, 1038, 1007, 4799, 7786, 1006, 1039, 1007, 5948, 13137, 1006, 1040, 1007, 22746, 12201, 102, 16991, 2965, 1996, 4338, 1997, 1037, 3430, 1025, 1997, 2019, 4874, 2064, 2022, 4089, 2904, 102]
Which of these is MOST flexible? (A) Broom handle (B) Wooden ruler (C) Drinking straw (D) Sewing needle [SEP] a drinking straw is flexible
[101, 2029, 1997, 2122, 2003, 2087, 12379, 1029,

In [39]:
text_q[1]

'Earth orbits the Sun once a year. About how many times does the moon orbit Earth in a year? (A) 1 (B) 7 (C) 13 (D) 28'

In [37]:
set_e['1a29-2268-eeb7-edba']

'the moon orbiting the Earth approximately occurs 13 times per year'

In [52]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [10]:
# build encoder
#module_url = "https://tfhub.dev/google/universal-sentence-encoder/2"
#embed = hub.Module(module_url)

## get embedding
#with tf.Session() as session:
#    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
#    question_embeddings = session.run(embed(text_q))
#    answer_embeddings = session.run(embed(text_e))

# define BERT to tokenize captions
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = BertModel.from_pretrained('bert-base-uncased')
#
## get emebedding
#question_embeddings = []
#answer_embeddings = []
#with torch.no_grad():
#    for single_q in text_q:
#        token_q = tokenizer.encode(single_q)
#        input_id = torch.tensor(token_q).unsqueeze(0)
#        last_hidden = bert(input_id)[0]
#        last_hidden_mean = torch.mean(last_hidden, dim=1)[0]
#        question_embeddings.append(last_hidden_mean.tolist())

#    for single_e in text_e:
#        token_e = tokenizer.encode(single_e)
#        input_id = torch.tensor(token_e).unsqueeze(0)
#        last_hidden = bert(input_id)[0]
#        last_hidden_mean = torch.mean(last_hidden, dim=1)[0]
#        answer_embeddings.append(last_hidden_mean.tolist())

#X_q = vectorizer.transform(df_q['Question'])
#X_e = vectorizer.transform(df_e['text'])
X_dist = cosine_distances(question_embeddings, answer_embeddings)

for i_question, distances in enumerate(X_dist):
    for i_explanation in np.argsort(distances)[:args.nearest]:
        print('{}\t{}'.format(df_q.loc[i_question]['questionID'], df_e.loc[i_explanation]['uid']))

/home/haoyu/.local/lib/python3.5/site-packages/ipykernel_launcher.py:54: UserWarning: Possibly misformatted file: data/annotation/expl-tablestore-export-2017-08-25-230344/tables/LemmatizerAdditions.tsv
/home/haoyu/.local/lib/python3.5/site-packages/ipykernel_launcher.py:54: UserWarning: Possibly misformatted file: data/annotation/expl-tablestore-export-2017-08-25-230344/tables/Decompositions.tsv
/home/haoyu/.local/lib/python3.5/site-packages/ipykernel_launcher.py:54: UserWarning: Possibly misformatted file: data/annotation/expl-tablestore-export-2017-08-25-230344/tables/PROCESSSTAGES.tsv


0             VASoL_2008_3_26
1           Mercury_SC_415491
2             VASoL_2007_3_33
3           Mercury_SC_415772
4           Mercury_SC_415366
5           Mercury_SC_416133
6           Mercury_SC_415026
7             VASoL_2008_3_39
8           Mercury_SC_416526
9           Mercury_SC_406705
10          Mercury_SC_415541
11          Mercury_SC_416172
12          Mercury_SC_416516
13          Mercury_SC_415764
14          Mercury_SC_416137
15          Mercury_SC_415348
16          Mercury_SC_415396
17          Mercury_SC_415354
18          Mercury_SC_415696
19             VASoL_2010_3_5
20          Mercury_SC_415397
21            VASoL_2007_3_32
22          Mercury_SC_416156
23          Mercury_SC_417580
24          Mercury_SC_413081
25          Mercury_SC_407784
26        Mercury_SC_LBS10920
27     NYSEDREGENTS_2014_4_20
28     NYSEDREGENTS_2014_4_19
29      NYSEDREGENTS_2014_4_8
                ...          
234         Mercury_SC_400194
235         Mercury_SC_401340
236       

NameError: name 'question_embeddings' is not defined